<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/earthengine-py-documentation/tree/master/Image/07_morphological_operations.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/earthengine-py-documentation/blob/master/Image/07_morphological_operations.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/earthengine-py-documentation/master?filepath=Image/07_morphological_operations.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/earthengine-py-documentation/blob/master/Image/07_morphological_operations.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Morphological Operations
Earth Engine implements morphological operations as focal operations, specifically `focal_max()`, `focal_min()`, `focal_median()`, and `focal_mode()` instance methods in the `Image` class. (These are shortcuts for the more general `reduceNeighborhood()`, which can input the pixels in a kernel to any reducer with a numeric output. See [this page](https://developers.google.com/earth-engine/reducers_reduce_neighborhood) for more information on reducing neighborhoods). The morphological operators are useful for performing operations such as erosion, dilation, opening and closing. For example, to perform an [opening operation](http://en.wikipedia.org/wiki/Opening_(morphology)), use `focal_min()` followed by `focal_max()`:

## Install Earth Engine API
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geehydro](https://github.com/giswqs/geehydro). The **geehydro** Python package builds on the [folium](https://github.com/python-visualization/folium) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, `Map.centerObject()`, and `Map.setOptions()`.
The following script checks if the geehydro package has been installed. If not, it will install geehydro, which automatically install its dependencies, including earthengine-api and folium.

In [1]:
import subprocess

try:
    import geehydro
except ImportError:
    print('geehydro package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geehydro'])

# Import libraries    
import ee
import folium
import geehydro

# Authenticate and initialize Earth Engine API
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map 
This step creates an interactive map using [folium](https://github.com/python-visualization/folium). The default basemap is the OpenStreetMap. Additional basemaps can be added using the `Map.setOptions()` function. 
The optional basemaps can be `ROADMAP`, `SATELLITE`, `HYBRID`, `TERRAIN`, or `ESRI`.

In [2]:
Map = folium.Map(location=[40, -100], zoom_start=4)
Map.setOptions('HYBRID')

## Add Earth Engine Python script 

In [3]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318') \
            .select(4).gt(0.2)
Map.setCenter(-122.1899, 37.5010, 13)
Map.addLayer(image, {}, 'NIR threshold')

# Define a kernel.
kernel = ee.Kernel.circle(**{'radius': 1})

# Perform an erosion followed by a dilation, display.
opened = image \
             .focal_min(**{'kernel': kernel, 'iterations': 2}) \
             .focal_max(**{'kernel': kernel, 'iterations': 2})
Map.addLayer(opened, {}, 'opened')

Map.setControlVisibility(layerControl=True, fullscreenControl=True, latLngPopup=True)
Map

Note that in the previous example, a kernel argument is provided to the morphological operator. The pixels covered by non-zero elements of the kernel are used in the computation. The iterations argument indicates how many times to apply the operator.